In [5]:
import pandas as pd
import numpy as np
import datetime
from faker import Faker
from customer import CustomerDataGeneration

In [371]:
class ProductInstanceGeneration:
    def __init__(self, customer_df, product_df, faker_locale):
        self.customer_df = customer_df
        self.product_df = product_df
        self.faker_locale = faker_locale
        self.fake = Faker(self.faker_locale)
        
        #Data block
        
        # ToDo
        self.business_product_instance_id  = [] # В конце всей генерации, т.к у пользователя может быть несколько инстансов
        # Useless part
        self.product_id = [] # Необходимо распределить тарифы и аддоны с логикой 
        self.activation_date = [] # Из Сustomer.data_since до текущего дня
        self.termination_date = [] # заполнять до статуса 
        self.Status = []
        self.distribution_channel = [] # В концце генерации вместе с инстанс id  
        # 
        self.product_instance_df = pd.DataFrame(columns = ["business_product_instance_id",
                                                          "customer_id",
                                                          "product_id",
                                                          "activation_date",
                                                          "termination_date",
                                                          "Status",
                                                          "distribution_channel"]) # Финальный этап
        #onwork
        
        #done
        # customer data
        #                                                    'ID',
        #                                                  'first_name',
        #                                                  'last_name',
        #                                                  'date_of_birth',
        #                                                  'gender',
        #                                                  'email',
        #                                                  'MSISDN',
        #                                                  'agree_for_promo',
        #                                                  'autopay_card',
        #                                                 'customer_category',
        #                                                 'language',
        #                                                 'customer_since',
        #                                                 'region',
        #                                                 'status']
        self.customer_id = list(self.customer_df.ID)
    
    def _tariff_generation(self, customer_id):
        product_type_tariff = self.product_df[self.product_df.product_type == "tariff"]
        customer = self.customer_df[self.customer_df["ID"] == customer_id]
        print(customer)
        tariff_list = []
        activation_date = []
        termination_date = []
        Status = []
        distribution_channel = []
        
        date_since_split = str(customer.customer_since.values[0]).split("-")
        customer_since = datetime.date(int(date_since_split[0]), int(date_since_split[1]), int(date_since_split[2])) # datetime.date(int(date_since_strip[0]),int(date_since_strip[1]),int(date_since_strip[2]))
        
        print(customer_since)
        if customer_since < datetime.date(2020,1,1): #LTE first
            tariff_list.append(np.random.choice(product_type_tariff[product_type_tariff.product_category == "LTE"].product_id))
            activation_date.append(self.fake.date_between(start_date=datetime.date(2010,1,1), end_date=datetime.date(2020,1,1)))
            Status.append("Active")
            distribution_channel.append(np.random.choice(["online","physical_shop","other"],p=[0.2,0.75,0.05]))
            termination_date.append(None)
            
            if np.random.choice([True,False],p=[0.8,0.2]): # Дописать логику для выбора termination и activation_date
                tariff_list.append(np.random.choice(product_type_tariff[product_type_tariff.product_id != tariff_list[0]].product_id))
                # Исправить ниже
                activation_date.append(self.fake.date_between(start_date=activation_date[0], end_date=datetime.date.today()))
                distribution_channel.append(np.random.choice(["online","physical_shop","other"],p=[0.75,0.2,0.05]))
                Status[0] = "Inactive"
                Status.append("Active")
                termination_date[0] = activation_date[1]
                termination_date.append(None)
                
                     
        elif customer_since > datetime.date(2020,1,1): #5G or LTE users
            tariff_list.append(np.random.choice(product_type_tariff.product_id))
            distribution_channel.append(np.random.choice(["online","physical_shop","other"],p=[0.75,0.2,0.05]))
            activation_date.append(customer_since)
            Status.append("Active")
            termination_date.append(None)
        return tariff_list,activation_date, termination_date,Status,distribution_channel  
        #return np.random.choice(product_type_tariff.product_id)
    
    def _random_addon_generation(self):
        
        product_type_addon = self.product_df[self.product_df.product_type == "addon"]
        return np.random.choice(product_type_tariff.product_id)
    
    def product_inst_for_customer(self,customer_id):
        tariff = self._tariff_generation(customer_id)
        print(self.product_instance_df.columns)
        for i in range(len(tariff[0])):
            
            instance = ["None",customer_id,tariff[0][i],tariff[1][i],tariff[2][i],tariff[3][i],tariff[4][i]]
            df_to_add = pd.DataFrame([instance],columns=self.product_instance_df.columns.values)
            
            self.product_instance_df = self.product_instance_df.append(df_to_add,ignore_index=True)
       
            # [["None"],[customer_id],[tariff[0][i]],[tariff[1][i]],[tariff[2][i]],[tariff[3][i]],["unknown"]]
            #self.product_instance_df["customer_id"][i] = customer_id
           # self.product_instance_df["product_id"][i] = tariff[0][i]
            #self.product_instance_df["activation_date"] = tariff[1][i]
            #self.product_instance_df["termination_date"] = tariff[2][i]
            #self.product_instance_df["Status"] = tariff[3][i]
    
        

In [372]:
pig = ProductInstanceGeneration(customer_df.head(10),product_df,"ja_JP")

In [373]:
%%time
for i in range(3):
    pig.product_inst_for_customer(i)

   ID first_name last_name date_of_birth gender                email  \
0   0         翔太        山本    1970-04-01      F  D*******6@yahoo.com   

         MSISDN agree_for_promo autopay_card customer_category  language  \
0  90-9534-3397             Yes           No          physical  Japanese   

  customer_since            region  status  
0     2002-04-01  Kyūshū & Okinawa  Active  
2002-04-01
Index(['business_product_instance_id', 'customer_id', 'product_id',
       'activation_date', 'termination_date', 'Status',
       'distribution_channel'],
      dtype='object')
   ID first_name last_name date_of_birth gender                  email  \
1   1          淳        山口    1957-02-17      M  H*******2@hotmail.com   

         MSISDN agree_for_promo autopay_card customer_category language  \
1  90-9030-5095             Yes           No          physical   Korean   

  customer_since region  status  
1     2020-10-03  Kantō  Active  
2020-10-03
Index(['business_product_instance_id', 'cust

In [374]:
pig.product_instance_df

,business_product_instance_id,customer_id,product_id,activation_date,termination_date,Status,distribution_channel
0,None,0,5,2014-03-21,2021-07-06,Inactive,physical_shop
1,None,0,1,2021-07-06,None,Active,other
2,None,1,3,2020-10-03,None,Active,online
3,None,2,13,2017-10-28,2018-05-26,Inactive,physical_shop
4,None,2,3,2018-05-26,None,Active,online


In [57]:
pig.product_id

array([ 5,  6,  3,  4,  3,  5, 13,  6,  3,  6], dtype=int64)

In [19]:
customer_df = pd.read_csv("Customer.csv",index_col = False)
customer_df

,ID,first_name,last_name,date_of_birth,gender,email,MSISDN,agree_for_promo,autopay_card,customer_category,language,customer_since,region,status
0,0,翔太,山本,1970-04-01,F,D*******6@yahoo.com,90-9534-3397,Yes,No,physical,Japanese,2002-04-01,Kyūshū & Okinawa,Active
1,1,淳,山口,1957-02-17,M,H*******2@hotmail.com,90-9030-5095,Yes,No,physical,Korean,2020-10-03,Kantō,Active
2,2,淳,石川,1954-08-14,M,R*******0@gmail.com,90-0250-0280,No,Yes,physical,Japanese,2016-03-23,Kantō,Active
3,3,浩,井上,1977-04-08,F,U*******8@hotmail.com,90-7439-0050,No,Yes,physical,Japanese,2013-01-18,Kyūshū & Okinawa,Active
4,4,幹,鈴木,1943-03-15,F,n*******9@yahoo.com,90-2473-2212,Yes,Yes,physical,Chinese,2013-11-08,Kantō,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,香織,加藤,2003-07-09,F,I*******8@hotmail.com,90-7408-8614,No,Yes,physical,Japanese,2021-02-22,Kansai(Kinki),Active
9996,9996,美加子,山田,1951-12-30,M,u*******5@yahoo.com,90-1219-8901,No,No,physical,Japanese,2020-06-27,Kansai(Kinki),Active
9997,9997,花子,石川,1943-11-16,M,K*******5@yahoo.com,90-7068-1582,No,No,physical,Japanese,2006-09-29,Kansai(Kinki),Active
9998,9998,くみ子,村上,1977-06-10,F,n*******6@yahoo.com,90-4017-7036,Yes,No,physical,Japanese,2016-07-20,Kansai(Kinki),Active


In [28]:
product_df = pd.read_csv("product.csv",index_col = False)
product_df

,product_id,product_name,product_category,product_type,recurrent,cost_for_call,cost_for_sms,cost_for_data,allowance_sms,allowance_voice,allowance_data,total_cost
0,1,5G Gigaho Premier,5G,tariff,regularly,22,3.3,0,NaN,NaN,unlimited,7315
1,2,Kake-hodai Option,5G,addon,regularly,0,0.0,0,NaN,unlimited,NaN,1870
2,3,5G Gigalight,5G,tariff,regularly,22,3.3,0,NaN,NaN,7,6765
3,4,Gigaho Premier,LTE,tariff,regularly,22,3.3,0,NaN,NaN,60,7205
4,5,Gigalight,LTE,tariff,regularly,22,3.3,0,NaN,NaN,7,6765
5,6,Hajimete Sumaho Plan,5G-LTE,tariff,regularly,0,3.3,0,NaN,NaN,1,1980
6,7,Kake-hodai Option (¥1000),5G-LTE,addon,regularly,0,0.0,0,NaN,unlimited,NaN,1100
7,8,U15 Hajimete Sumaho Plan,5G-LTE,tariff,regularly,22,3.3,0,NaN,NaN,5,1980
8,9,5G Data Plus,data only (5G),addon,regularly,0,0.0,0,NaN,NaN,30,1100
9,10,Data Plus,data only (LTE),addon,regularly,0,0.0,0,NaN,NaN,30,1100


In [255]:
product_type_addon = product_df[product_df.product_type == "addon"]
product_type_addon   

,product_id,product_name,product_category,product_type,recurrent,cost_for_call,cost_for_sms,cost_for_data,allowance_sms,allowance_voice,allowance_data,total_cost
1,2,Kake-hodai Option,5G,addon,regularly,0,0.0,0,NaN,unlimited,NaN,1870
6,7,Kake-hodai Option (¥1000),5G-LTE,addon,regularly,0,0.0,0,NaN,unlimited,NaN,1100
8,9,5G Data Plus,data only (5G),addon,regularly,0,0.0,0,NaN,NaN,30,1100
9,10,Data Plus,data only (LTE),addon,regularly,0,0.0,0,NaN,NaN,30,1100
10,11,LTE Jouku Riyou Plan,data only (LTE),addon,regularly,0,0.0,0,NaN,NaN,120,49800


In [214]:
print(datetime.date(2020,1,1))

2020-01-01


In [201]:
int("03")

3

In [ ]:
LTE Jouku Riyou Plan	data only (LTE)	addon	regularly	0	0.0	0	NaN	NaN	120	49800